In [1]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell, seq2seq
from tqdm import *
from tensorflow.python.ops import control_flow_ops
import sys
import random
import time

from Utils import Utils
import ncnn_lstm as model
from hyperboard import Agent

    
def train(train_utils, valid_utils, source_len, oseq_len, n_echos, simplified_len, decoder_hidden,
          embedding_size, batch_size, display_step, prob, lstm_layer, learning_rate,
          model_dir, source_nfilters, defendant_nfilters, source_width, defendant_width):
            
    
    hyperparameters = {
        'learning rate': learning_rate,
        'batch size': batch_size,
        'criteria': '',
        'decoder hidden': decoder_hidden,
        'prob': prob,
        'source_nfilters': source_nfilters,
        'defendant_nfilters': defendant_nfilters, 
        'source_width': source_width,
        'defendant_width': defendant_width, 
        'lstm_layer': lstm_layer,  
        'embedding_size': embedding_size
    }
        
    agent = Agent(port = 5002)
    
    hyperparameters['criteria'] = 'valid loss'
    name_valid_loss = agent.register(hyperparameters, 'cross entropy')
    
    hyperparameters['criteria'] = 'valid accu'
    name_valid_accu = agent.register(hyperparameters, 'accuracy')
    
    hyperparameters['criteria'] = 'train loss'
    name_train_loss = agent.register(hyperparameters, 'cross entropy')
    
    hyperparameters['criteria'] = 'train accu'
    name_train_accu = agent.register(hyperparameters, 'accuracy')
    
        
    if not os.path.isdir(model_dir):
        os.mkdir(model_dir)
        
    words_size = train_utils.get_words_size()
    
    outputs = model.build_model(words_size=words_size, 
                                embedding_size=embedding_size,
                                source_len=source_len,
                                simplified_len=simplified_len,
                                oseq_len=oseq_len, 
                                decoder_hidden=decoder_hidden,
                                source_nfilters=source_nfilters,
                                defendant_nfilters=defendant_nfilters,
                                source_width=source_width,
                                defendant_width=defendant_width,
                                lstm_layer=lstm_layer, 
                                batch_size=batch_size,  
                                is_train=True)

    cost=outputs['cost']
    words_prediction=outputs['words_prediction']
    source=outputs['source']
    defendant=outputs['defendant']
    defendant_length=outputs['defendant_length']
    label=outputs['label']
    decoder_inputs=outputs['decoder_inputs']
    loss_weights=outputs['loss_weights']
    keep_prob=outputs['keep_prob']
    sample_rate=outputs['sample_rate']
    
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='adam_optimizer')
    # Compute the gradients for a list of variables.
    grads_and_vars = optimizer.compute_gradients(cost)
    # grads_and_vars is a list of tuples (gradient, variable).
    capped_grads_and_vars = [(tf.clip_by_norm(g, 5), v) for g,v in grads_and_vars]
    # Ask the optimizer to apply the capped gradients.
    train_op = optimizer.apply_gradients(capped_grads_and_vars)

    
    init = tf.initialize_all_variables()

    os.environ["CUDA_VISIBLE_DEVICES"] = '1'
    gpu_option = tf.GPUOptions(per_process_gpu_memory_fraction = 0.1)
    session_conf = tf.ConfigProto(allow_soft_placement = True, 
                                  log_device_placement = False,
                                  gpu_options = gpu_option)
    
    sess = tf.Session()
    
#     pre_saver = tf.train.Saver()
#     ckpt = tf.train.latest_checkpoint('/home/xuwenshen/2017_3_13/2cnn-lstm/model_v1/')
#     pre_saver.restore(sess, ckpt)
    
    global_steps = 0
    
    echos = 0
    batch = 0
    
    saver = tf.train.Saver(max_to_keep = 20)

    sess.run(init)
    
    tvar = tf.trainable_variables()
    for v in tvar:
        print (v.name)
    
    print ('init done')
    
    train_cost = 0
    train_accu = 0
    train_bleu = 0
    
    while True:
        
        datas, is_again = train_utils.next_batch()
        
        if is_again:
            
            train_cost = 0
            train_accu = 0
            echos += 1
            batch = 0
            if echos == n_echos:
                break
            continue
            
        
        batch_source = datas['source'] 
        batch_defendant = datas['defendant'] 
        batch_defendant_length = datas['defendant_length']
        batch_ground_truth = datas['ground_truth']
        batch_label = datas['label']
        batch_weights = datas['loss_weights']
        
        batch += 1
        

        sample_rate_ = min(.5, 0.01 * (global_steps))

        feed_dic = {source:batch_source,
                    defendant:batch_defendant,
                    label:batch_label,
                    defendant_length:batch_defendant_length,
                    decoder_inputs:batch_ground_truth,
                    loss_weights:batch_weights,
                    keep_prob:prob,
                    sample_rate:sample_rate_}
        
        words_, cost_, _= sess.run([words_prediction, cost, train_op], feed_dict= feed_dic)
#         accu_, bleu_ = train_utils.i2t(words_, to_print = False)
    
        train_cost += cost_
#         train_accu += accu_
#         train_bleu += bleu_
        

        
        if batch % display_step == 0:
            
            valid_cost = 0
            valid_accu = 0
            valid_bleu = 0
            valid_batchs = 0
            
            train_cost /= display_step
            train_accu /= display_step
            train_bleu /= display_step
            
            while True:
                
                datas, is_again = valid_utils.next_batch()
                
                if is_again:
                    break
        
                batch_source = datas['source'] 
                batch_defendant = datas['defendant'] 
                batch_defendant_length = datas['defendant_length']
                batch_ground_truth = datas['ground_truth']
                batch_label = datas['label']
                batch_weights = datas['loss_weights']

                valid_batchs += 1
                
                feed_dic = {source:batch_source,
                            defendant:batch_defendant,
                            label:batch_label,
                            defendant_length:batch_defendant_length,
                            decoder_inputs:batch_ground_truth,
                            loss_weights:batch_weights,
                            keep_prob:prob,
                            sample_rate:sample_rate_}
                    
                word_, cost_, = sess.run([words_prediction, cost], feed_dict=feed_dic)     
                accu_, bleu_ = valid_utils.i2t(word_, to_print = True)
                
                valid_cost += cost_
                valid_accu += accu_
                valid_bleu += bleu_
            
            valid_cost /= valid_batchs
            valid_accu /= valid_batchs
            valid_bleu /= valid_batchs
            
            saver.save(sess, model_dir + \
                       "sample_rate-" + str(sample_rate_) + \
                       "-train_accu-{:.3f}".format(train_accu) + \
                       "-train_cost-{:.3f}".format(train_cost) + \
                       "-train_bleu-{:.3f}".format(train_bleu) + \
                       "-valid_accu-{:.3f}".format(valid_accu) + \
                       "-valid_cost-{:.3f}".format(valid_cost) + \
                       "-valid_bleu-{:.3f}".format(valid_bleu) + \
                       "-model.ckpt", global_step = global_steps)
            
            
            print ("Echo: "+str(echos) + " Iters: "+str(batch) + \
                   " Sample: " + "{:.3f}".format(sample_rate_) + \
                   " Train loss: " + "{:.3f}".format(train_cost) + \
                   " Valid loss: " + "{:.3f}".format(valid_cost) + \
                   " Train accu: " + "{:.3f}".format(train_accu) + \
                   " Valid accu: " + "{:.3f}".format(valid_accu) + \
                   " Train bleu: " + "{:.3f}".format(train_bleu) + \
                   " Valid bleu: " + "{:.3f}".format(valid_bleu))
            

            agent.append(name_valid_loss, global_steps, valid_cost)
            agent.append(name_valid_accu, global_steps, valid_accu)
            agent.append(name_train_loss, global_steps, train_cost)
            agent.append(name_train_accu, global_steps, train_accu)
            
            global_steps += 1
        
            
if __name__ == '__main__':
    
    words_path = '/home/xuwenshen/data/big_data/2017_3_13/words'
    train_path = '/home/xuwenshen/data/big_data/2017_3_13/train.h5'
    valid_path = '/home/xuwenshen/data/big_data/2017_3_13/valid.h5'
    
    oseq_len = 200
    source_len = 1000
    simplified_len = 150

    batch_size = 50
    display_step = 400
    n_echos = 100
    prob = 0.75
    decoder_hidden = 750
    source_nfilters = 480
    defendant_nfilters = 75
    source_width = 3
    defendant_width = 3
    lstm_layer = 1
    learning_rate = 0.001
    embedding_size = 200

    
    train_utils = Utils(words_path, train_path, batch_size, nb_samples=1600000)
    valid_utils = Utils(words_path, valid_path, batch_size, nb_samples=640)

    train(train_utils=train_utils,
          valid_utils=valid_utils,
          source_len=source_len,
          simplified_len=simplified_len,
          oseq_len=oseq_len,
          embedding_size=embedding_size,
          batch_size=batch_size,
          display_step=display_step,
          n_echos=n_echos,
          prob=prob,
          decoder_hidden=decoder_hidden,
          source_nfilters=source_nfilters,
          defendant_nfilters=defendant_nfilters,
          source_width=source_width,
          defendant_width=defendant_width,
          lstm_layer=lstm_layer,
          learning_rate=learning_rate,
          model_dir='/home/xuwenshen/2017_3_13/2cnn-lstm/model_v1/')
    
    
    

[None, 998, 1, 480]
[None, 996, 1, 480]
[None, 994, 1, 480]
[None, 148, 1, 75]
[None, 146, 1, 75]
[None, 144, 1, 75]
build model 
embedding:0
conv_source_w_1:0
conv_source_w_2:0
conv_source_w_3:0
conv_source_b_1:0
conv_source_b_2:0
conv_source_b_3:0
conv_source_scale_1:0
conv_source_scale_2:0
conv_source_scale_3:0
conv_source_offset_1:0
conv_source_offset_2:0
conv_source_offset_3:0
conv_defendant_w_1:0
conv_defendant_w_2:0
conv_defendant_w_3:0
conv_defendant_b_1:0
conv_defendant_b_2:0
conv_defendant_b_3:0
conv_defendant_scale_1:0
conv_defendant_scale_2:0
conv_defendant_scale_3:0
conv_defendant_offset_1:0
conv_defendant_offset_2:0
conv_defendant_offset_3:0
generation_w:0
generation_b:0
rnn_decoder/BasicLSTMCell/Linear/Matrix:0
rnn_decoder/BasicLSTMCell/Linear/Bias:0
init done
公诉机关吉林省敦化市人民检察院。被告人张某某。因涉嫌犯危险驾驶罪，于2013年12月31日被取保候审。吉林省敦化市人民检察院以敦检公诉刑诉（2014）24号起诉书指控被告人张某某犯危险驾驶罪，于2014年2月12日向本院提起公诉。本院依法适用简易程序，实行独任审判，公开开庭审理了本案。敦化市人民检察院指派检察员严志红出庭支持公诉，被告人张某某到庭参加诉讼。现已审理终结。公诉机关指控：2013年12月17日15时50分许，在敦

/home/xuwenshen/py3/lib/python3.4/site-packages/nltk/translate/bleu_score.py:472: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


公诉机关大连市沙河口区人民检察院。被告人刘某某，男，汉族，高中文化，无职业，住辽宁省盘锦市双台子区。因涉嫌犯盗窃罪于2014年6月28日经大连市公安局沙河口分局决定被刑事拘留，同年7月30日经大连市沙河口区人民检察院批准逮捕，并于次日由该分局执行逮捕。现羁押于大连市看守所。大连市沙河口区人民检察院以沙检刑诉（2014）571号起诉书指控被告人刘某某犯盗窃罪，于2014年9月23日向本院提起公诉。本院于当日立案并依法适用简易程序，实行独任审判，于2014年9月28日公开开庭审理了本案。大连市沙河口区人民检察院代理检察员周育、被告人刘某某到庭参加诉讼。现已审理终结。公诉机关指控：被告人刘某某于2014年6月2日15时许，在本市沙河口区敦煌南街70号&ldquo;洪源水果超市&rdquo;办公室内，趁杨娜离开之机，盗窃被害人杨娜放在办公桌抽屉内的人民币8000元，&ldquo;三星&rdquo;牌9500手机1部，价值人民币1680元。被告人刘某某于2014年6月13日17时许，在本市沙河口区沙能街32号&ldquo;神州融合印刷店&rdquo;内，趁吴某某复印材料之机，盗窃被害人吴某某放在桌子上的&ldquo;苹果&rdquo;牌4S手机1部，价值人民币1840元。上述事实，被告人刘某某在开庭审理过程中亦无异议，且有被害人杨某、吴某某陈述、发货票、鉴定意见、视听资料以及案件来源、抓获经过、常住人口基本信息表等证据证实，足以认定。
--------------------

被告人刘某某，571号起诉书指控被告人刘某某犯盗窃罪，公诉机关指控：被告人刘某某于2014年6月2日15时许，被告人刘某某于2014年6月13日17时许，被告人刘某某在开庭审理过程中亦无异议，因涉嫌犯盗窃罪于2014年6月28日经大连市公安局沙河口分局决定被刑事拘留，大连市沙河口区人民检察院代理检察员周育、被告人刘某某到庭参加诉讼，
--------------------

本院认为，被告人刘某某以非法占有为目的，秘密窃取公民财物，数额较大，其行为侵犯了公民财产所有权，构成盗窃罪，应予刑罚处罚。大连市沙河口区人民检察院指控被告人刘某某犯盗窃罪的事实清楚，证据确实、充分，指控的罪名成立
--------------------

本院认为，被告人刘某某在非法占有为目的，秘密窃取他诉事物，其额较大，其行

KeyboardInterrupt: 